# Port

> TODO fill in description

In [1]:
#| default_exp ports

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio
from abc import ABC, abstractmethod
from typing import Type, Callable, Any
import inspect
from collections import deque

import fbdev
from fbdev import Packet
from fbdev.packet_container import PacketContainer, SinglePacketContainer, DequePacketContainer

In [5]:
#|export
__all__ = [
    'BasePort', 'InputPort', 'OutputPort'
]

In [6]:
#|hide
show_doc(fbdev.BasePort)

---

### BasePort

>      BasePort (name:str, data_validator:Callable[[Any],bool])

*Initialize self.  See help(type(self)) for accurate signature.*

In [7]:
#|export
class BasePort:
    def __init__(self, name:str, data_validator:Callable[[Any], bool]):
        self.name = name
        self.idx = None
        self._component_process = None
        self.data_validator = data_validator
        
    def attach(self, component_process, port_index:int):
        if self._component_process is not None:
            #TODO proper exception in BasePort.attach
            raise Exception("Port is already attached.")
        self._component_process = component_process
        self.idx = port_index
            
    async def _validate_packet(self, packet):
        if self.data_validator is not None:
            payload = await packet.get_payload()
            if not self.data_validator(payload):
                #TODO logging in BasePort._validate_packet_data
                #TODO proper exceptions in BasePort._validate_packet_data
                #TODO unit test BasePort._validate_packet_data
                raise Exception("Packet payload is invalid.")

In [8]:
#|hide
show_doc(fbdev.InputPort)

---

### InputPort

>      InputPort (name:str, data_validator:Callable[[Any],bool]=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [9]:
#|export
class InputPort(BasePort):
    def __init__(self, name:str, data_validator:Callable[[Any], bool]=None):
        super().__init__(name, data_validator)
        self._packet = None
        self._requesting_package = asyncio.Event() # Set when the port is requesting a package
        self._received_package = asyncio.Event() # Set when the port has returned a package to the component process
        
    async def receive():
        self._requesting_package.set()
        await self._received_package.wait()
        self._requesting_package.clear()
        self._received_package.clear()
        packet = self._packet
        self._packet = None
        await self._validate_packet(packet)
        return packet

In [10]:
#|hide
show_doc(fbdev.OutputPort)

---

### OutputPort

>      OutputPort (name:str, data_validator:Callable[[Any],bool]=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [11]:
#|export
class OutputPort(BasePort):
    def __init__(self, name:str, data_validator:Callable[[Any], bool]=None):
        super().__init__(name, data_validator)
        self._packet = None
        self._ready_to_send = asyncio.Event() # Set when the port is requesting to send out a packet
        self._package_sent = asyncio.Event() # Set when the port has returned a package to the component process
        
    async def put(packet):
        if type(packet) != Packet: raise TypeError("`packet` must be of type Packet.")
        await self._validate_packet(packet)
        self._packet = packet
        self._ready_to_send.set()
        await self._package_sent.wait()
        self._ready_to_send.clear()
        self._package_sent.clear()